In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import folium

from src.cred import *
from src.get_data import get_activity,get_summary
from src.prepare_data import import_activity

pd.set_option("display.max_columns",1000)

In [2]:
summary = get_summary(mail,pw)
summary.head(2)

,activeLengths,activeSets,activityId,activityLikeAuthors,activityLikeDisplayNames,activityLikeFullNames,activityName,activityTrainingLoad,activityType,aerobicTrainingEffect,aerobicTrainingEffectMessage,anaerobicTrainingEffect,anaerobicTrainingEffectMessage,atpActivity,autoCalcCalories,averageBikingCadenceInRevPerMinute,averageHR,averageRunningCadenceInStepsPerMinute,averageSpeed,averageSwimCadenceInStrokesPerMinute,averageSwolf,avgAirSpeed,avgCda,avgDepth,avgDoubleCadence,avgFlow,avgFractionalCadence,avgGrit,avgGroundContactBalance,avgGroundContactTime,avgLeftBalance,avgPower,avgRespirationRate,avgStress,avgStrideLength,avgStrokeCadence,avgStrokeDistance,avgStrokes,avgVerticalOscillation,avgVerticalRatio,avgVerticalSpeed,avgWattsPerCda,avgWindYawAngle,beginTimestamp,bottomTime,calories,caloriesConsumed,caloriesEstimated,commentedByUser,comments,conversationPk,conversationUuid,courseId,decoDive,description,deviceId,differenceStress,distance,diveNumber,duration,elapsedDuration,elevationCorrected,elevationGain,elevationLoss,endCns,endLatitude,endLongitude,endN2,endStress,eventType,excludeFromPowerCurveReports,favorite,floorsClimbed,floorsDescended,flow,grit,hasPolyline,hasSplits,hasVideo,intensityFactor,jumpCount,lactateThresholdBpm,lactateThresholdSpeed,lapCount,leftBalance,likedByUser,locationName,manualActivity,manufacturer,max20MinPower,maxAirSpeed,maxAvgPower_1,maxAvgPower_10,maxAvgPower_120,maxAvgPower_1200,maxAvgPower_1800,maxAvgPower_18000,maxAvgPower_2,maxAvgPower_20,maxAvgPower_30,maxAvgPower_300,maxAvgPower_3600,maxAvgPower_5,maxAvgPower_60,maxAvgPower_600,maxAvgPower_7200,maxBikingCadenceInRevPerMinute,maxCda,maxDepth,maxDoubleCadence,maxElevation,maxFractionalCadence,maxFtp,maxHR,maxPower,maxRespirationRate,maxRunningCadenceInStepsPerMinute,maxSpeed,maxStress,maxStrokeCadence,maxSwimCadenceInStrokesPerMinute,maxTemperature,maxVerticalSpeed,minActivityLapDuration,minAirSpeed,minCda,minElevation,minRespirationRate,minStrokes,minTemperature,moderateIntensityMinutes,movingDuration,normPower,numberOfActivityComments,numberOfActivityLikes,ownerDisplayName,ownerFullName,ownerId,ownerProfileImageUrlLarge,ownerProfileImageUrlMedium,ownerProfileImageUrlSmall,parent,parentId,poolLength,pr,privacy,purposeful,requestorRelationship,rightBalance,splitSummaries,sportTypeId,startCns,startLatitude,startLongitude,startN2,startStress,startTimeGMT,startTimeLocal,steps,strokes,summarizedDiveInfo,summarizedExerciseSets,surfaceInterval,timeZoneId,totalReps,totalSets,trainingEffectLabel,trainingStressScore,unitOfPoolLength,userPro,userRoles,vO2MaxValue,videoUrl,vigorousIntensityMinutes,waterConsumed,waterEstimated,workoutId
0,NaN,None,5989547573,None,None,None,Saint-Pol-sur-Ternoise Course,None,"{'typeId': 1, 'typeKey': 'running', 'parentTyp...",3.1,IMPROVING_LACTATE_THRESHOLD_12,1.7,MINOR_ANAEROBIC_BENEFIT_15,False,False,NaN,154.0,172.359375,3.515,NaN,NaN,None,None,NaN,None,None,None,None,NaN,NaN,NaN,NaN,None,None,122.133627,None,NaN,NaN,NaN,NaN,NaN,None,None,1.608649e+12,NaN,878.0,None,None,None,None,None,None,NaN,None,None,3.968818e+09,None,14210.790039,None,4043.224121,4.165973e+06,False,147.0,158.0,None,50.384037,2.348362,None,None,"{'typeId': 9, 'typeKey': 'uncategorized', 'sor...",None,False,None,NaN,None,None,True,False,False,NaN,None,NaN,NaN,15.0,None,None,Saint-Pol-sur-Ternoise,False,GARMIN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,197.0,15100.000000,None,NaN,186.0,NaN,None,197.0,5.011,None,None,NaN,None,0.800003,115.874001,None,None,8480.000305,None,None,15.0,None,4039.073975,NaN,None,None,ixxxen,ixxxen,1141258,https://s3.amazonaws.com/garmin-connect-prod/p...,https://s3.amazonaws.com/garmin-connect-prod/p...,https://s3.amazonaws.com/garmin-connect-prod/p...,False,None,NaN,False,"{'typeId': 1, 'typeKey': 'public'}",False,None,None,[],1.0,None,50.380486,2.343743,None,None,2020-12-22 15:02:19,2020-12-22 16:02:19,11578.0,NaN,"{'weight': None, 'weightUnit': None, 'visibili...",None,NaN,124,None,

In [3]:
act = import_activity(mail,pw,"5989547573")
act.head(2)

Parsing file


,altitudefeet,altitudemeters,cadence,cadencex2,distancekilometers,distancemeters,distancemiles,elapsedtime,heartratebpm,latitudedegrees,longitudedegrees,seq,speed,time,type
0,349.081370,106.400002,80,160.0,0.00000,0.00,0.000000,00:00:00,86,50.380486,2.343743,1,3.219,2020-12-22 15:02:19+00:00,Trackpoint
1,349.737528,106.599998,92,184.0,0.00234,2.34,0.001454,00:00:01,86,50.380484,2.343776,2,2.697,2020-12-22 15:02:20+00:00,Trackpoint


In [20]:
px.scatter(act,x="longitudedegrees",y="latitudedegrees",color="altitudemeters",hover_name="distancemeters",
            color_continuous_scale='viridis')

In [19]:
fig = px.line_mapbox(act,lon="longitudedegrees",lat="latitudedegrees",zoom=10)
fig.update_layout(mapbox_style="open-street-map",margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [16]:
#[tuple(x) for x in act[["longitudedegrees","latitudedegrees"]].to_numpy()]
points = list(act[["latitudedegrees","longitudedegrees"]].itertuples(index=False))
myMap = folium.Map([act["latitudedegrees"].mean(),act["longitudedegrees"].mean()])
folium.PolyLine(points).add_to(myMap)
myMap